In [35]:
import torch
import numpy as np
import operator
import math

In [5]:
x = torch.empty(5, 3)
print(x)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


In [6]:
x = torch.rand(5, 3)
print(x)

tensor([[0.2012, 0.7006, 0.0685],
        [0.8754, 0.0235, 0.1121],
        [0.8929, 0.1105, 0.0917],
        [0.0814, 0.0635, 0.8166],
        [0.9105, 0.9210, 0.0853]])


In [8]:
x = torch.zeros(5, 3, dtype = torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [9]:
x = torch.tensor([5.5, 3])
print(x)

tensor([5.5000, 3.0000])


In [10]:
x = x.new_ones(5, 3)
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])


In [16]:
x.size()

torch.Size([5, 3])

In [18]:
y = torch.rand(5, 3)
result = torch.empty(5, 3)
torch.add (x, y, out = result)

tensor([[1.4262, 1.0661, 1.0662],
        [1.6769, 1.0176, 1.9241],
        [1.8122, 1.8169, 1.8032],
        [1.3747, 1.8122, 1.2546],
        [1.7174, 1.9286, 1.4377]])

In [19]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)
z

tensor([[-1.0202, -1.1443, -0.1051,  2.2824,  0.3574, -0.1758, -0.8805,  1.7793],
        [ 1.0300, -0.0786, -0.3269,  0.2332,  0.0331,  0.1424,  0.3752,  0.3183]])

CUDA Tensors

In [21]:
print(torch.cuda.is_available())
if torch.cuda.is_available():
    device = torch.device("cuda")
    y = torch.ones_like(x, device = device)
    x = x.to(device)
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))

False


In [22]:
#tensor可以在gpu上，但numpy只有cpu，所以一般都是np转为tensor放到gpu运算，然后再转回numpy并放到cpu里
y.to("cpu").data.numpy()
y.cpu().data.numpy()

array([-1.0201975 , -1.1442864 , -0.10506658,  2.2824323 ,  0.3573812 ,
       -0.17584214, -0.88049424,  1.7792723 ,  1.0299932 , -0.07858746,
       -0.32689717,  0.23323706,  0.03314139,  0.1423604 ,  0.3752281 ,
        0.31828701], dtype=float32)

用numpy实现两层神经网络
----------------
一个全连接ReLU神经网络，一个隐藏层，没有bias。用来从x预测y，使用L2 Loss
- h = W_1X + b_1
- a = max (0, h) ReLU
- y_{hat} = W_2a + b_2

实现
- forward pass
- loss
- backward pass

numpy ndarray只是一个普通的n维array，并不涉及任何深度学习或者梯度gradient的知识。只是一种用来计算数学运算的数据结构

In [37]:
N, D_in, H, D_out = 64, 1000, 100, 10
#64个输入，1000维度，hidden是100维度，输出10维度
#64个训练数据一个batch，把一个1000维度的向量转成一个10维度的向量

#随机创建一些训练数据
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = math.pow(10, -6)

for it in range(50):
    #forward pass
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)
    
    #compute loss
    #MSE mean square error
    loss = np.square(y_pred - y).sum()
    print(it, loss)
    
    #backward pass
    #compute the gradient
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)
    
    #update weights of w1 and w2
    w1 -= learning_rate * grad_w1
    w2 = w2 - learning_rate * grad_w2

0 37330079.9821741
1 31141540.215710815
2 28895152.73736138
3 25755639.577937238
4 20276683.309540555
5 13961857.319894487
6 8637280.106683142
7 5148183.258450883
8 3140292.6277650776
9 2049105.3749066382
10 1445036.9350677927
11 1091533.661590245
12 866827.2291749764
13 712188.8905861593
14 598114.3194623955
15 509185.82570199104
16 437904.52453823196
17 379485.29022308456
18 330720.57711180503
19 289628.576905093
20 254734.85746594996
21 224932.21133046155
22 199274.61230761284
23 177080.16045152151
24 157808.08275309845
25 141028.7806726255
26 126345.90107098638
27 113441.09583735152
28 102065.27420329615
29 92010.4478221159
30 83098.91305846235
31 75190.52280873308
32 68157.42504827632
33 61875.923908946854
34 56261.48556820596
35 51228.48500897418
36 46708.78706976758
37 42641.244425472934
38 38976.83110366805
39 35670.40327172875
40 32683.089063409698
41 29979.16442706629
42 27527.978337167668
43 25301.599765721796
44 23277.07860867127
45 21435.1011697373
46 19756.95286878866
47 